In [18]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [ ]:
# class GCNLayer(nn.Module):
#     def __init__(self, in_feats, out_feats):
#         super(GCNLayer, self).__init__()
#         self.linear = nn.Linear(in_feats, out_feats)

#     def forward(self, g, feature):
#         # Creating a local scope so that all the stored ndata and edata
#         # (such as the `'h'` ndata below) are automatically popped out
#         # when the scope exits.
#         with g.local_scope():
#             g.ndata['h'] = feature
#             g.update_all(gcn_msg, gcn_reduce)
#             h = g.ndata['h']
#             return self.linear(h)

In [2]:
from dgl.nn.pytorch import GraphConv

class GCN(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        super(GCN, self).__init__()
        self.g = g
        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(GraphConv(in_feats, n_hidden, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(GraphConv(n_hidden, n_hidden, activation=activation))
        # output layer
        self.layers.append(GraphConv(n_hidden, n_classes))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, features):
        h = features
        for i, layer in enumerate(self.layers):
            if i != 0:
                h = self.dropout(h)
            h = layer(self.g, h)
        return h

In [3]:
import time
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import register_data_args
from dgl.data import CoraGraphDataset

In [4]:
data = CoraGraphDataset()

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [5]:
def evaluate(model, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [6]:
g = data[0]

In [7]:
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = data.graph.number_of_edges()
print("""----Data statistics------'
  #Edges %d
  #Classes %d
  #Train samples %d
  #Val samples %d
  #Test samples %d""" %
      (n_edges, n_classes,
          train_mask.int().sum().item(),
          val_mask.int().sum().item(),
          test_mask.int().sum().item()))

----Data statistics------'
  #Edges 10556
  #Classes 7
  #Train samples 140
  #Val samples 500
  #Test samples 1000


C:\Users\ME\Anaconda3\envs\tf-gpu5\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
C:\Users\ME\Anaconda3\envs\tf-gpu5\lib\site-packages\dgl\data\utils.py:285: UserWarning: Property dataset.graph will be deprecated, please use dataset.g instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [8]:
n_hidden = 16
n_layers = 1
dropout = 0.5
weight_decay = 5e-4
lr = 1e-2

In [9]:
n_edges = g.number_of_edges()

# normalization
degs = g.in_degrees().float()
norm = torch.pow(degs, -0.5)
norm[torch.isinf(norm)] = 0

g.ndata['norm'] = norm.unsqueeze(1)

In [19]:
model = GCN(g,
            in_feats,
            n_hidden,
            n_classes,
            n_layers,
            F.relu,
            dropout)

In [20]:
initial_weight = model.layers[0].weight

In [21]:
initial_weight

Parameter containing:
tensor([[-0.0478,  0.0296, -0.0427,  ...,  0.0005, -0.0385,  0.0307],
        [ 0.0244, -0.0186,  0.0104,  ...,  0.0459, -0.0055, -0.0459],
        [ 0.0012, -0.0006, -0.0235,  ...,  0.0482,  0.0159,  0.0404],
        ...,
        [-0.0318, -0.0606,  0.0195,  ..., -0.0441, -0.0218, -0.0516],
        [-0.0455,  0.0088, -0.0339,  ..., -0.0614,  0.0523,  0.0169],
        [-0.0205,  0.0078, -0.0483,  ..., -0.0221,  0.0318,  0.0300]],
       requires_grad=True)

In [22]:
loss_fcn = torch.nn.CrossEntropyLoss()
# use optimizer
optimizer = torch.optim.Adam(model.parameters(),
                     lr=lr,
                     weight_decay=weight_decay)

In [23]:
dur = []
n_epochs = 200

for epoch in range(n_epochs):
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    logits = model(features)
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, features, labels, val_mask)
    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
          "ETputs(KTEPS) {:.2f}". format(epoch, np.mean(dur), loss.item(),
                                         acc, n_edges / np.mean(dur) / 1000))

Epoch 00000 | Time(s) nan | Loss 1.9466 | Accuracy 0.2240 | ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 1.9411 | Accuracy 0.2380 | ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 1.9377 | Accuracy 0.2700 | ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.0209 | Loss 1.9301 | Accuracy 0.3980 | ETputs(KTEPS) 504.01
Epoch 00004 | Time(s) 0.0219 | Loss 1.9200 | Accuracy 0.5220 | ETputs(KTEPS) 481.07
Epoch 00005 | Time(s) 0.0226 | Loss 1.9111 | Accuracy 0.5520 | ETputs(KTEPS) 466.93
Epoch 00006 | Time(s) 0.0247 | Loss 1.9044 | Accuracy 0.5460 | ETputs(KTEPS) 427.62
Epoch 00007 | Time(s) 0.0249 | Loss 1.8952 | Accuracy 0.5260 | ETputs(KTEPS) 423.32
Epoch 00008 | Time(s) 0.0271 | Loss 1.8827 | Accuracy 0.5280 | ETputs(KTEPS) 389.57
Epoch 00009 | Time(s) 0.0276 | Loss 1.8704 | Accuracy 0.5160 | ETputs(KTEPS) 381.88
Epoch 00010 | Time(s) 0.0281 | Loss 1.8586 | Accuracy 0.5260 | ETputs(KTEPS) 376.32
Epoch 00011 | Time(s) 0.0280 | Loss 1.8475 | Accuracy 0.5620 | ETputs(KTEPS) 376.51
Epoch 0001

Epoch 00100 | Time(s) 0.0274 | Loss 0.6142 | Accuracy 0.7700 | ETputs(KTEPS) 385.40
Epoch 00101 | Time(s) 0.0274 | Loss 0.5575 | Accuracy 0.7760 | ETputs(KTEPS) 384.62
Epoch 00102 | Time(s) 0.0275 | Loss 0.5527 | Accuracy 0.7780 | ETputs(KTEPS) 384.14
Epoch 00103 | Time(s) 0.0274 | Loss 0.6021 | Accuracy 0.7780 | ETputs(KTEPS) 384.90
Epoch 00104 | Time(s) 0.0274 | Loss 0.5804 | Accuracy 0.7760 | ETputs(KTEPS) 385.52
Epoch 00105 | Time(s) 0.0274 | Loss 0.5897 | Accuracy 0.7680 | ETputs(KTEPS) 384.63
Epoch 00106 | Time(s) 0.0275 | Loss 0.5767 | Accuracy 0.7680 | ETputs(KTEPS) 383.63
Epoch 00107 | Time(s) 0.0275 | Loss 0.5559 | Accuracy 0.7700 | ETputs(KTEPS) 383.64
Epoch 00108 | Time(s) 0.0275 | Loss 0.5903 | Accuracy 0.7660 | ETputs(KTEPS) 384.11
Epoch 00109 | Time(s) 0.0275 | Loss 0.5577 | Accuracy 0.7700 | ETputs(KTEPS) 384.05
Epoch 00110 | Time(s) 0.0275 | Loss 0.5837 | Accuracy 0.7680 | ETputs(KTEPS) 383.35
Epoch 00111 | Time(s) 0.0275 | Loss 0.5762 | Accuracy 0.7680 | ETputs(KTEPS)

In [14]:
print()
acc = evaluate(model, features, labels, test_mask)
print("Test accuracy {:.2%}".format(acc))


Test accuracy 80.10%


In [34]:
dir(model.layers[0])

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activation',
 '_allow_zero_in_degree',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_forward_unimplemented',
 '_get_name',
 '_in_feats',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_norm',
 '_out_feats',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_m

In [24]:
final_weight = model.layers[0].weight

In [25]:
final_weight

Parameter containing:
tensor([[ 6.8080e-02, -1.6991e-02, -9.5129e-03,  ..., -4.8230e-03,
          2.2167e-03,  8.0389e-07],
        [ 1.2927e-01, -1.0263e-01, -3.5900e-01,  ..., -2.6162e-01,
         -9.1564e-02, -6.6161e-07],
        [ 6.9595e-03, -4.3922e-01, -1.8890e-01,  ..., -1.9583e-01,
         -1.0577e-01, -3.5050e-07],
        ...,
        [-7.6585e-03, -7.9028e-02, -5.4745e-03,  ..., -1.8528e-02,
          2.9665e-02, -3.7492e-07],
        [-1.2457e-02,  7.9623e-02,  1.0097e-01,  ...,  6.0082e-02,
          1.3345e-01, -8.2251e-07],
        [-6.7522e-02,  1.9789e-01,  9.2289e-02,  ...,  1.4291e-01,
          1.1179e-02, -2.5974e-07]], requires_grad=True)